In [ ]:
## predict chirps on unlabelled data
unlabelled_folder = Path(r'./young_chirps_unlabelled/')
unlabelled_audio = list(unlabelled_folder.glob("*.flac"))

In [ ]:
import collections
import datetime
def run_cough_detector(cough_detector, single_audio_file):
    n_mel = 17 
    date_time_obj = datetime.datetime.strptime(str(single_audio_file)[35:54], '%Y-%m-%d_%H.%M.%S')
    date_index = int(str(test_file)[43:45])
                                               
    signal = AudioSignal.from_file(single_audio_file)
    features = calc_log_mel_energy_features(signal,n_mels = n_mel,
                                            window_length_seconds = 0.06)
    features_summed = np.sum(features.features, axis=1)
    features.event_names = ["cough"]
    train_features = features.features
    y_pred = y_pred = [bool(c) for c in cough_detector.predict(train_features)]

    labels_dataframe = pd.DataFrame(
        {
            "onset": pd.Series(features.frame_start_times[y_pred]),
            "offset": pd.Series(features.frame_end_times[y_pred]),
            "event_label": pd.Series("chirp", index=range(np.sum(y_pred))),
        }
    )
    labels_dataframe = clean_overlapping_labels(labels_dataframe)
    c = collections.Counter(list(labels_dataframe.iloc[:,-1]))
    return date_index,date_time_obj,c["chirp"]

In [ ]:
clock_unlabelled = {7:{},8:{}}
for item in unlabelled_audio:
    date_index,date_time_obj,num_chirps= run_cough_detector(knn_chirp,item)
    clock_unlabelled[date_index][date_time_obj] = num_chirps

In [ ]:
# padding zeros for data that we are not predicting
for mins in range(44):
    hours = mins // 60
    minutes = mins % 60
    date_time_obj = datetime.datetime.strptime("2020-02-08_"+str(hours)+"."+str(minutes)+'.30', 
                                                   '%Y-%m-%d_%H.%M.%S');
    clock_unlabelled[8][date_time_obj] = 0
    
for mins in range(121,1261):
    hours = mins // 60
    minutes = mins % 60
    date_time_obj = datetime.datetime.strptime("2020-02-08_"+str(hours)+"."+str(minutes)+'.30', 
                                                   '%Y-%m-%d_%H.%M.%S');
    
for mins in range(1380,1420):
    hours = mins // 60
    minutes = mins % 60
    date_time_obj = datetime.datetime.strptime("2020-02-08_"+str(hours)+"."+str(minutes)+'.30', 
                                                   '%Y-%m-%d_%H.%M.%S');
    clock_unlabelled[8][date_time_obj] = 0
clock_unlabelled

In [ ]:
# unlabelled data
dfn = pd.DataFrame.from_dict(clock_unlabelled[8], orient='index')
dfn.sort_index(inplace=True)
dfn.index.name = 'time'
dfn.reset_index(inplace=True)
dfn.columns = ['time', 'counts']
dfn.head()

In [ ]:
## removing all other labeled data to this notebook
## couting chirps label in the labeled data
chirp_train_folder = Path(r'./young_chirps/')
chirp_train_labels_list = list(chirp_train_folder.glob("*.txt"))

chirp_test_folder = Path(r'./young_chirps_test/')
chirp_test_labels_list = list(chirp_test_folder.glob("*.txt"))

In [ ]:
import collections
import datetime
print("young chirps folder...")
clock = {7:{},8:{}}

for mins in range(1260,1440):
    hours = mins // 60
    minutes = mins % 60
    date_time_obj = datetime.datetime.strptime("2020-02-07_"+str(hours)+"."+str(minutes)+'.30', 
                                                   '%Y-%m-%d_%H.%M.%S');
    clock[7][date_time_obj] = 0

for mins in range(1440):
    hours = mins // 60
    minutes = mins % 60
    date_time_obj = datetime.datetime.strptime("2020-02-08_"+str(hours)+"."+str(minutes)+'.30', 
                                                   '%Y-%m-%d_%H.%M.%S');
    clock[8][date_time_obj] = 0
        
for _, y_path in enumerate(chirp_train_labels_list+chirp_test_labels_list):
#     print(y_path)
    path = str(y_path).replace('young_chirps_test', 'young_chirps')
    times = str(path[35:40])
    time_index = 60*int(times[0:2])+int(times[3:])
    date_time_obj = datetime.datetime.strptime(path[24:43], '%Y-%m-%d_%H.%M.%S')
    times = date_time_obj
    date_index = int(path[32:34])
    
    try:# in case there are empty files
        label_data = pd.read_csv(y_path, sep="\t", header=None)
        c = collections.Counter(list(label_data.iloc[:,-1]))
        clock[date_index][times] = c["chirp"]
    except: 
        clock[date_index][times] = 0

In [ ]:
df7 = pd.DataFrame.from_dict(clock[7], orient='index')
df7.sort_index(inplace=True)
df7.index.name = 'time'
df7.reset_index(inplace=True)
df7.columns = ['time', 'counts']

In [ ]:
df8 = pd.DataFrame.from_dict(clock[8], orient='index')
df8.sort_index(inplace=True)
df8.index.name = 'time'
df8.reset_index(inplace=True)
df8.columns = ['time', 'counts']

In [ ]:
import matplotlib.pyplot as plt

xmin = datetime.datetime.strptime("2020-02-08_"+'21'+"."+'00'+'.30', 
                                                   '%Y-%m-%d_%H.%M.%S')
xmax = datetime.datetime.strptime("2020-02-08_"+'23'+"."+'59'+'.30', 
                                                   '%Y-%m-%d_%H.%M.%S')
ymin = 0
ymax = 250
ax = df8.plot(x = 'time', y = 'counts',color = 'green', figsize = (30,5),x_compat=True)
ax.set(xlim=(xmin, xmax), ylim=(ymin, ymax))

df7.plot(x = 'time', y = 'counts',color = 'green', ax=ax)
dfn.plot(x = 'time', y = 'counts',color = 'blue', ax=ax)
plt.title("chirp counts at the night for 2020-02-08",fontsize = 20)
#plt.xticks(np.arange(min(xmin), max(xmax)+1))
ax.get_legend().remove()

plt.show()